In [1]:
import cv2

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile,Image
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os

In [12]:
train_folder = "Dataset/Dataset/Train" 
test_folder = "Dataset/Dataset/Test"
dimensions = (50, 50)

In [13]:
def make_greyscale_white_bg(im, r, b, g):

    im = im.convert('RGBA')   # Convert to RGBA


    data = np.array(im)   # "data" is a height x width x 4 numpy array
    red, green, blue, alpha = data.T # Temporarily unpack the bands for readability

    # Replace grey with white... (leaves alpha values alone...)
    grey_areas = (red == r) & (blue == b) & (green == g)
    data[..., :-1][grey_areas.T] = (255, 255, 255) # Transpose back needed

    im2 = Image.fromarray(data)
    im2 = im2.convert('L')   # convert to greyscale image


    return im2


In [14]:
def invert_colors(im):

    im = im.convert('RGBA')   # Convert to RGBA
    data = np.array(im)   # "data" is a height x width x 4 numpy array
    red, green, blue, alpha = data.T # Temporarily unpack the bands for readability


    # Replace black with red temporarily... (leaves alpha values alone...)
    black_areas = (red == 0) & (blue == 0) & (green == 0)
    data[..., :-1][black_areas.T] = (255, 0, 0) # Transpose back needed

    # Replace white areas with black
    white_areas = (red == 255) & (blue == 255) & (green == 255)
    data[..., :-1][white_areas.T] = (0, 0, 0) # Transpose back needed

    # Replace red areas (originally white) with black
    red_areas = (red == 255) & (blue == 0) & (green == 0)
    data[..., :-1][red_areas.T] = (255, 255, 255) # Transpose back needed

    im2 = Image.fromarray(data)
    im2 = im2.convert('L')   # convert to greyscale image


    return im2


In [17]:
def process_images(folder):

    classes = [os.path.join(folder, d) for d in sorted(os.listdir(folder))]  # get list of all sub-folders in folder
    img_cnt = 0

    for class_x in classes:

        if os.path.isdir(class_x):

            # get paths to all the images in this folder
            images = [os.path.join(class_x, i) for i in sorted(os.listdir(class_x)) if i != '.DS_Store']

            for image in images:

                img_cnt = img_cnt + 1

                if(img_cnt % 1000 == 0):                # show progress
                    print("Processed %s images" % str(img_cnt))

                im = Image.open(image)
                print(image)
                im = im.resize(dimensions)   # resize image according to dimensions set

                im = make_greyscale_white_bg(im, 127, 127, 127) # turn grey background (if any) to white, and                                                 # convert into greyscale image with 1 channel
                  # overwrite previous image file with new image
                im = invert_colors(im)
                im.save(image)
    print("Finished processing images, images found = ")
    print(img_cnt)
process_images(test_folder)

Dataset/Dataset/Test\172\bcc000000.bmp
Dataset/Dataset/Test\172\bcc000001.bmp
Dataset/Dataset/Test\172\bcc000002.bmp
Dataset/Dataset/Test\172\bcc000003.bmp
Dataset/Dataset/Test\172\bcc000004.bmp
Dataset/Dataset/Test\172\bcc000005.bmp
Dataset/Dataset/Test\172\bcc000006.bmp
Dataset/Dataset/Test\172\bcc000007.bmp
Dataset/Dataset/Test\172\bcc000008.bmp
Dataset/Dataset/Test\172\bcc000009.bmp
Dataset/Dataset/Test\172\bcc000010.bmp
Dataset/Dataset/Test\172\bcc000011.bmp
Dataset/Dataset/Test\172\bcc000012.bmp
Dataset/Dataset/Test\172\bcc000013.bmp
Dataset/Dataset/Test\172\bcc000014.bmp
Dataset/Dataset/Test\172\bcc000015.bmp
Dataset/Dataset/Test\172\bcc000016.bmp
Dataset/Dataset/Test\172\bcc000017.bmp
Dataset/Dataset/Test\172\bcc000018.bmp
Dataset/Dataset/Test\172\bcc000019.bmp
Dataset/Dataset/Test\172\bcc000020.bmp
Dataset/Dataset/Test\172\bcc000021.bmp
Dataset/Dataset/Test\172\bcc000022.bmp
Dataset/Dataset/Test\172\bcc000023.bmp
Dataset/Dataset/Test\172\bcc000024.bmp
Dataset/Dataset/Test\172\

UnidentifiedImageError: cannot identify image file 'Dataset/Dataset/Test\\172\\bcc000060.db'

In [ ]:
def process_images(folder):

    classes = [os.path.join(folder, d) for d in sorted(os.listdir(folder))]  # get list of all sub-folders in folder
    img_cnt = 0

    for class_x in classes:

        if os.path.isdir(class_x):

            # get paths to all the images in this folder
            images = [os.path.join(class_x, i) for i in sorted(os.listdir(class_x)) if i != '.DS_Store']

            for image in images:

                img_cnt = img_cnt + 1

                if(img_cnt % 1000 == 0):                # show progress
                    print("Processed %s images" % str(img_cnt))
                img_bgr = cv2.imread(image)  # Read image as BGR

                img_grey = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)  # Convert from BGR to grayscale.

                thresh = 170
                img_binary_inv = cv2.threshold(img_grey, thresh, 255, cv2.THRESH_BINARY_INV)[1]  # Apply threshold and invert black/white

                # Convert from BGR to HSV color space.
                hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

                # Get the saturation color channel - all black and white are zero, and colored pixels are above zero.
                s = hsv[:, :, 1]

                thresh = 100
                s_binary = cv2.threshold(s, thresh, 255, cv2.THRESH_BINARY)[1]  # Apply threshold to the saturation channel.

                # Find contours on s_binary
                cnts = cv2.findContours(s_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2]  # Use index [-2] to be compatible to OpenCV 3 and 4

                # Draw the contours as white (255 values) on black background.
                mask = np.zeros_like(s_binary)
                cv2.drawContours(mask, cnts, -1, 255, -1)

                # Apply morphological closing for closing some black gaps.
                mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((3, 3)))

                # Get only the area inside the mask from img_binary_inv
                masked_binary_inv = cv2.bitwise_or(img_binary_inv, img_binary_inv, mask=mask)

                # Copy masked_binary_inv to img_grey only in pixels that mask is white.
                cv2.copyTo(masked_binary_inv, mask, img_grey)

                cv2.imwrite(image, img_grey)  # Save result (as PNG and not JPEG for better quality).

                # Show images
                cv2.imshow('img_bgr', img_bgr)
                cv2.imshow('s_binary', s_binary)
                cv2.imshow('mask', mask)
                cv2.imshow('masked_binary_inv', masked_binary_inv)
                cv2.imshow('img_grey', img_grey)
                cv2.waitKey()
                cv2.destroyAllWindows() # turn grey background (if any) to white, and                                                 # convert into greyscale image with 1 channel
                                  # overwrite previous image file with new image
    print("Finished processing images, images found = ")
    print(img_cnt)

process_images(test_folder)